In [ ]:
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np

## Bootstrapping demo

### Percentiles

In [ ]:
# Manually compute the 55th percentile.
x = make_array(43, 20, 51, 7, 28, 34)
x

In [ ]:
# Step 1. Sort the data
np.sort(x)

In [ ]:
# Step 2. Figure out where 55th percentile would be.

In [ ]:
# OR: 1 Line of Code
percentile(55, x)

In [ ]:
# If we tried to compute which element to take...
55 / 100 * 6

### Sample Median

In [ ]:
sf = Table.read_table('https://raw.githubusercontent.com/lujiec2020/UMBC-IS296-Fall2022/main/data/san_francisco_2015.csv')
sf
sf.num_rows

In [ ]:
# Who is making the most money
sf.sort('Total Compensation', descending=True).show(5)

In [ ]:
# Who is making the least money
sf.sort('Total Compensation', descending=False).show(5)

In [ ]:
min_salary = 10 * 20 * 52
sf = sf.where('Total Compensation', are.above(min_salary))
sf.num_rows/42989

In [ ]:
pop_median = percentile(50, sf.column('Total Compensation'))
pop_median

In [ ]:
our_sample = sf.sample(300, with_replacement=False)
#our_sample.show(5)
percentile(50, our_sample.column('Total Compensation'))

In [ ]:
sf_bins = np.arange(0, 700000, 25000)
sf.hist('Total Compensation', bins=sf_bins)
plots.title('Population Distribution');

In [ ]:
our_sample.hist('Total Compensation', bins=sf_bins)
plots.title('Sample Distribution');

# Variability of the Estimate

In [ ]:
def generate_sample_median(samp_size):
    our_sample = sf.sample(samp_size, with_replacement=False)
    return percentile(50, our_sample.column('Total Compensation'))

In [ ]:
sample_median = generate_sample_median(300)
sample_median

In [ ]:
error = sample_median - pop_median
error

# Quantifying Uncertainty

In [ ]:
sample_medians = make_array()

for i in np.arange(1000):
    new_median = generate_sample_median(300)
    sample_medians = np.append(sample_medians, new_median)

In [ ]:
len(sample_medians)

In [ ]:
med_bins = np.arange(90000, 125001, 2500)
Table().with_column(
    'Sample Medians', sample_medians
).hist(bins = med_bins)

plots.scatter(pop_median, 0.00005, color="red", s=400);

In [ ]:
err_bins = np.arange(-15000, 12501, 2500)
Table().with_column(
    'Errors', sample_medians - pop_median
).hist(bins = err_bins)

plots.scatter(0, 0, color="red");

# Bootstrap

In [ ]:
# Take a bootstrap (re)sample of size 300, WITH replacement
boot_sample = our_sample.sample()
boot_sample.hist('Total Compensation', bins=sf_bins)
plots.title('Bootstrap sample');

print("Population Median =       ", pop_median)
print("Our Sample Median =       ", sample_median)
print("Bootstrap Sample Median = ", 
      percentile(50,boot_sample.column('Total Compensation')))

In [ ]:
def one_bootstrap_median():
    single_sample = our_sample.sample()
    return percentile(50, single_sample.column('Total Compensation'))
    #return np.mean(single_sample.column('Total Compensation'))
    

In [ ]:
bootstrap_medians = make_array()
for i in np.arange(10000):
    new_median = one_bootstrap_median()
    bootstrap_medians = np.append(bootstrap_medians, new_median)
bootstrap_medians

In [ ]:
Table().with_column(
    'Bootstrap Medians', bootstrap_medians
).hist('Bootstrap Medians', bins=med_bins)

plots.scatter(pop_median, 0, color="red", s=1000);
plots.scatter(sample_median, 0, color="blue",s=1000);

In [ ]:
len(bootstrap_medians)

## Confidence Intervals

In [ ]:
# Make an interval based on the middle 95% of bootstrap samples

left = percentile(2.5, bootstrap_medians)
right = percentile(97.5, bootstrap_medians)

Table().with_column(
    'Bootstrap Medians', bootstrap_medians
).hist('Bootstrap Medians', bins=med_bins)

plots.plot([left, right], [0,0], color="gold",lw=20, zorder=1);
plots.scatter(pop_median, 0, color="red", zorder=2);
plots.scatter(sample_median, 0, color="blue", zorder=2);